In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle
import torch

In [26]:
data = pd.read_csv("./spambase_csv.csv").dropna()
data = shuffle(data)

labels = data["class"]
unique_labels = labels.unique()
for i in unique_labels:
    print(sum(labels == i))

labels = np.array(labels).reshape(-1,1)
features = data.drop(["class"],axis=1)
features = ss().fit_transform(features)


1813
2788


In [27]:
labels = torch.tensor(labels,dtype = torch.float32)
features = torch.tensor(np.array(features),dtype = torch.float32)
print(features.shape,labels.shape)

torch.Size([4601, 57]) torch.Size([4601, 1])


In [28]:
model = torch.nn.Sequential(
    torch.nn.Linear(57,40),
    torch.nn.ReLU(),
    torch.nn.Linear(40,20),
    torch.nn.ReLU(),
    torch.nn.Linear(20,10),
    torch.nn.ReLU(),
    torch.nn.Linear(10,1),
    torch.nn.Sigmoid()
)

preds = model(features)

In [29]:
loss_fn = torch.nn.BCELoss()
opt= torch.optim.SGD(model.parameters(),lr=0.01)

In [30]:
from torch import no_grad
epochs=10000
for epoch in range(epochs):
    preds=model(features)
    loss=loss_fn(preds,labels)
    loss.backward()
    opt.step()
    opt.zero_grad()
    
    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            for i in range(len(preds)):
                if preds[i][0] >= 0.5:
                    preds[i][0] = 1
                else:
                    preds[i][0] = 0

                if preds[i][0] == labels[i][0]:
                    right += 1
            print(f"Loss : {loss}")
            print(f"Accuracy :",round(right * 100/ len(preds), 2),"%")

Loss : 0.6634271144866943
Accuracy : 60.6 %
Loss : 0.4503747820854187
Accuracy : 85.76 %
Loss : 0.205284982919693
Accuracy : 93.26 %
Loss : 0.16401498019695282
Accuracy : 94.04 %
Loss : 0.1459115445613861
Accuracy : 94.91 %
Loss : 0.13236390054225922
Accuracy : 95.31 %
Loss : 0.12047551572322845
Accuracy : 95.54 %
Loss : 0.10993407666683197
Accuracy : 95.98 %
Loss : 0.10009505599737167
Accuracy : 96.37 %
Loss : 0.09107634425163269
Accuracy : 96.78 %


In [31]:
torch.save(model.state_dict(),"SpamEmail_weights.pth")

In [32]:

pred = model(features[150])
i = torch.where(pred == pred.max())
print(pred,i)

tensor([7.2474e-12], grad_fn=<SigmoidBackward0>) (tensor([0]),)
